# Who can still win the F1 WDC between all drivers?

This notebook calculate which drivers still has chance to win the WDC. Is simplified because it doesn't compare positions if points are equal.


In [13]:
import fastf1
from fastf1.ergast import Ergast
import requests

In this case, we are looking at current (2024) season. We want to know who can still win the WDC after Monza race (actually the last race).


In [14]:
# Setting up season and round
SEASON = 2024
ROUND = 16  # Monza Race

Here we retrieve driver standings from Ergast API.


In [15]:
# Function to retrieve current driver standings
def getDriversStandings():
    ergast = Ergast()
    standings = ergast.get_driver_standings(season=SEASON, round=ROUND)
    return standings.content[0]

This function below we calculate how many points are left for the remaining season.


In [16]:
# Calculate how many points are left for the remaining season
def calculateMaxPointsForRemaingSeason():
    POINTS_FOR_SPRINT = 8 + 25 + 1
    # Winning the sprint race, Sunday race and do the fastest lap
    POINTS_CONVENTIONAL_RACE = 25 + 1  # Winning the Sunday race and do the fastest lap

    events = fastf1.events.get_event_schedule(SEASON, backend="fastf1")
    events = events[events["RoundNumber"] > ROUND]

    # Count how many sprint and conventional races are left
    sprintRace = len(events.loc[events["EventFormat"] == "sprint_shootout"])
    conventionalRace = len(events.loc[events["EventFormat"] == "conventional"])

    # Calculate points for each event
    sprintPoints = sprintRace * POINTS_FOR_SPRINT
    conventionalPoints = conventionalRace * POINTS_CONVENTIONAL_RACE

    return sprintPoints + conventionalPoints

This function below calculates who can still win the WDC.


In [17]:
# Calculate who can still win the WDC
def calculateDriverWhoCanWin(driverStandings, maxPoints):
    LEADER_POINTS = int(driverStandings.loc[0]["points"])

    for i, _ in enumerate(driverStandings.iterrows()):
        driver = driverStandings.loc[i]
        driverMaxPoints = int(driver["points"]) + maxPoints
        canWin = "No" if driverMaxPoints < LEADER_POINTS else "Yes"

        print(
            f"{driver['position']}: {driver['givenName'] + ' ' + driver['familyName']}, "
            f"Current points: {driver['points']}, "
            f"Theoretical max points: {driverMaxPoints}, "
            f"Can win: {canWin}"
        )

Here we can call function to predict who can still win the WDC.


In [ ]:
# Call the functions to predict who can still win the WDC

# Get the current drivers standings
driver_standings = getDriversStandings()

# Get the maximum amount of points
points = calculateMaxPointsForRemaingSeason()

# Print which drivers can still win
calculateDriverWhoCanWin(driver_standings, points)